# Deeplearning4j
+ Импорт данных
+ Конфигурация нейронной сети
+ TrainingMaster
+ Обучение
+ Результаты

In [1]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.deeplearning4j.nn.conf.NeuralNetConfiguration
import org.deeplearning4j.nn.conf.layers.{DenseLayer, OutputLayer}
import org.deeplearning4j.nn.weights.WeightInit
import org.deeplearning4j.optimize.listeners.ScoreIterationListener
import org.deeplearning4j.spark.impl.multilayer.SparkDl4jMultiLayer
import org.deeplearning4j.spark.impl.paramavg.ParameterAveragingTrainingMaster
import org.nd4j.linalg.activations.Activation
import org.nd4j.linalg.lossfunctions.LossFunctions
import org.deeplearning4j.spark.util.MLLibUtil.fromLabeledPoint
import org.apache.spark.mllib.linalg.Vectors

import org.nd4j.Nd4jRegistrator

# Импорт данных
Создадим RDD с LabeledPoint

In [2]:
val data = (sc.textFile("/user/supp.bda08/iris.csv")
    .filter(_ != "0,1,2,3,0")
    .map(_.split(","))
    .map(row => {
        val vector = row.slice(0,4).map(_.toDouble)
        val label  = row(4).toInt
        LabeledPoint(label, Vectors.dense(vector))
    }))

data.first

Waiting for a Spark session to start...

data = MapPartitionsRDD[4] at map at <console>:43


(0.0,[5.1,3.5,1.4,0.2])

Разделим выборку на обучение и тест

In [3]:
val splited = data.randomSplit(weights = Array(0.65, 0.35), seed = 11)

val train = splited(0)
val test = splited(1)

splited = Array(MapPartitionsRDD[5] at randomSplit at <console>:42, MapPartitionsRDD[6] at randomSplit at <console>:42)
train = MapPartitionsRDD[5] at randomSplit at <console>:42
test = MapPartitionsRDD[6] at randomSplit at <console>:42


MapPartitionsRDD[6] at randomSplit at <console>:42

# Конфигурация нейронной сети
Зададим конфигурацию и алгоритм обучения нейронной сети

In [4]:
val numInputs = 4
val outputNum = 3
val iterations = 1000
val seed = 6

val conf = (new NeuralNetConfiguration.Builder()
  .seed(seed).iterations(iterations)
  .activation(Activation.TANH)
  .weightInit(WeightInit.XAVIER)
  .learningRate(0.1)
  .regularization(true).l2(1e-4)
  .list
  .layer(0, new DenseLayer.Builder().nIn(numInputs).nOut(3).build)
  .layer(1, new DenseLayer.Builder().nIn(3).nOut(3).build)
  .layer(2, new OutputLayer.Builder(LossFunctions.LossFunction.NEGATIVELOGLIKELIHOOD)
    .activation(Activation.SOFTMAX).nIn(3).nOut(outputNum).build)
  .backprop(true).pretrain(false)
  .build)

numInputs = 4
outputNum = 3
iterations = 1000
seed = 6
conf = 


        "lea...


{
  "backprop" : true,
  "backpropType" : "Standard",
  "confs" : [ {
    "iterationCount" : 0,
    "l1ByParam" : { },
    "l2ByParam" : { },
    "layer" : {
      "dense" : {
        "activationFn" : {
          "TanH" : { }
        },
        "adamMeanDecay" : "NaN",
        "adamVarDecay" : "NaN",
        "biasInit" : 0.0,
        "biasLearningRate" : 0.1,
        "dist" : null,
        "dropOut" : 0.0,
        "epsilon" : "NaN",
        "gradientNormalization" : "None",
        "gradientNormalizationThreshold" : 1.0,
        "l1" : 0.0,
        "l1Bias" : 0.0,
        "l2" : 1.0E-4,
        "l2Bias" : 0.0,
        "layerName" : "layer0",
        "learningRate" : 0.1,
        "learningRateSchedule" : null,
        "momentum" : "NaN",
        "momentumSchedule" : null,
        "nin" : 4,
        "nout" : 3,
        "rho" : "NaN",
        "rmsDecay" : "NaN",
        "updater" : "SGD",
        "weightInit" : "XAVIER"
      }
    },
    "leakyreluAlpha" : 0.0,
    "learningRateByParam" 

# TrainingMaster
TrainingMaster задает опции, необходимые для распределенного обучения

In [5]:
val batchSizePerWorker = 16
val tm = (new ParameterAveragingTrainingMaster.Builder(batchSizePerWorker)
  .averagingFrequency(5)
  .workerPrefetchNumBatches(2)
  .batchSizePerWorker(batchSizePerWorker)
  .build)

batchSizePerWorker = 16
tm = ParameterAveragingTrainingMaster(saveUpdater=false, numWorkers=null, rddDataSetNumExamples=16, batchSizePerWorker=16, averagingFrequency=5, prefetchNumBatches=2, collectTrainingStats=false, stats=null, iterationCount=0, repartition=Always, repartitionStrategy=Balanced, storageLevel=StorageLevel(false, true, false, false, 1), storageLevelStreams=StorageLevel(false, true, false, true, 1), rddTrainingApproach=Export, exportDirectory=null, rng=java.util.Random@240d2366, trainingHookList=null, lastExportedRDDId=-2147483648, lastRDDExportPath=null, trainingMasterUID=1508325876828_60379857, listeners=null, statsStorage=null)


ParameterAveragingTrainingMaster(saveUpdater=false, numWorkers=null, rddDataSetNumExamples=16, batchSizePerWorker=16, averagingFrequency=5, prefetchNumBatches=2, collectTrainingStats=false, stats=null, iterationCount=0, repartition=Always, repartitionStrategy=Balanced, storageLevel=StorageLevel(false, true, false, false, 1), storageLevelStreams=StorageLevel(false, true, false, true, 1), rddTrainingApproach=Export, exportDirectory=null, rng=java.util.Random@240d2366, trainingHookList=null, lastExportedRDDId=-2147483648, lastRDDExportPath=null, trainingMasterUID=1508325876828_60379857, listeners=null, statsStorage=null)

# Обучение
Создадим объект SparkDl4jMultiLayer, который принимает SparkContext, конфигурацию нейронной сети и TrainingMaster.

In [6]:
val sparkNet = new SparkDl4jMultiLayer(sc, conf, tm)
sparkNet.setListeners(new ScoreIterationListener(100))

val trainRDD = fromLabeledPoint(train, 3, train.count.toInt)
trainRDD.cache()

sparkNet = org.deeplearning4j.spark.impl.multilayer.SparkDl4jMultiLayer@7ec9903f
trainRDD = MapPartitionsRDD[11] at flatMap at MLLibUtil.java:240


MapPartitionsRDD[11] at flatMap at MLLibUtil.java:240

Проведем обучение на 10 эпохах

In [7]:
val epochs = 10
for (i <- 1 to epochs) {
  sparkNet.fit(trainRDD)
}

epochs: Int = 10


# Результаты
Посмотрим на результаты модели на тестовой выборке

In [8]:
val eval = sparkNet.evaluate(fromLabeledPoint(test, 3))
println(eval.stats())
tm.deleteTempFiles(sc)

eval = org.deeplearning4j.eval.Evaluation@b15dc9d


true


Examples labeled as 0 classified by model as 0: 17 times
Examples labeled as 1 classified by model as 1: 16 times
Examples labeled as 1 classified by model as 2: 3 times
Examples labeled as 2 classified by model as 2: 21 times


==========================Scores========================================
 Accuracy:        0.9474
 Precision:       0.9583
 Recall:          0.9474
 F1 Score:        0.9528
